###  想办法识别出下面这几张图片中的细线方框, 程序要能够得到细线方框的四个坐标, 在控制台打印出来

方案<br>
这个好一点
一定要消噪！

In [2]:
import cv2
import numpy as np

image_path = 'photo/1.jpg'

image = cv2.imread(image_path)
# 消噪
denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    
# 图像增强和对比度增强
alpha = 1.5  # 对比度增强参数（默认为1.0）
beta = 15  # 亮度增强参数
enhanced_image = cv2.convertScaleAbs(denoised_image, alpha=alpha, beta=beta)

# 边缘检测
edges = cv2.Canny(enhanced_image, 50, 150)

# 查找轮廓
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#新建轮廓列表
quadrilaterals = []
for contour in contours:
    # 计算轮廓的面积
    area = cv2.contourArea(contour)
    # 排除面积较小的轮廓
    if area > 2000:
        # 计算轮廓的周长
        perimeter = cv2.arcLength(contour, True)
        # 对轮廓进行多边形逼近
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

        # 如果逼近的轮廓是四边形，将其添加到列表中
        if len(approx) == 4:
            quadrilaterals.append(approx)

marked_image = image.copy()

for quad in quadrilaterals:
    # 绘制轮廓
    cv2.drawContours(marked_image, [quad], -1, (200, 250, 10), 2)

    # 绘制顶点并标记坐标
    for i, point in enumerate(quad):
        x, y = point[0]
        cv2.circle(marked_image, (x, y), 5, (0, 255, 255), -1)
        cv2.putText(marked_image, f"({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 50, 255), 2)

    # 计算中心点并标记坐标
    M = cv2.moments(quad)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])

cv2.circle(marked_image, (cx, cy), 5, (0, 255, 0), -1)
cv2.putText(marked_image, f"({cx}, {cy})", (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

cv2.namedWindow("Marked Image", cv2.WINDOW_NORMAL)
cv2.imshow("Marked Image", marked_image)


#保存测试图片到相应区域
#（文件夹/文件名）
save_path = 'finish/1/square.jpg'
cv2.imwrite(save_path, marked_image)



cv2.waitKey(0)
cv2.destroyAllWindows()

### 红色和绿色

###  

In [4]:
import cv2
import numpy as np


# 读取图像
image = cv2.imread('photo/1.jpg')


# 将图像转换为HSV颜色空间
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定义红色和绿色的HSV范围
lower_red = np.array([165, 100, 100])
upper_red = np.array([179, 255, 255])

lower_green = np.array([36, 100, 100])
upper_green = np.array([86, 255, 255])

# 根据颜色范围创建掩模
mask_red = cv2.inRange(hsv, lower_red, upper_red)
mask_green = cv2.inRange(hsv, lower_green, upper_green)

# 寻找红色点的轮廓
contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 寻找绿色点的轮廓
contours_green, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 遍历红色点的轮廓
for contour in contours_red:
    # 获取红色点的坐标
    #（x，y）为轮廓最左边的坐标
    #z为x向右移z个单位（即是轮廓的宽）
    #k为y向下移k个单位（即是轮廓的高）
    x, y, z, k = cv2.boundingRect(contour)
    # 在图像上标出绿色区域的中心坐标
    cv2.putText(image, f'Red: ({x + z // 2}, {y + k // 2})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    print("红色点坐标:", x + z // 2, y + k // 2)
# 遍历绿色点的轮廓
for contour in contours_green:
    # 获取绿色点的坐标
    #（x，y）为轮廓最左边的坐标
    #z为x向右移z个单位（即是轮廓的宽）
    #k为y向下移k个单位（即是轮廓的高）
    x, y, z, k = cv2.boundingRect(contour)
    # 在图像上标出绿色区域的中心坐标
    cv2.putText(image, f'Green: ({x + z // 2}, {y + k // 2})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    print("绿色点坐标:", x + z // 2, y + k // 2)

# 显示带有红色点和绿色点的图像
cv2.namedWindow('红绿色点', cv2.WINDOW_NORMAL)
cv2.imshow('红绿色点', image)



#保存测试图片到相应区域
#（文件夹/文件名）
save_path = 'finish/1/color.jpg'
cv2.imwrite(save_path, image)



cv2.waitKey(0)
cv2.destroyAllWindows()

红色点坐标: 480 357
绿色点坐标: 964 413


使用色差分析和角度检测（针对图4）

In [ ]:
# 如果轮廓有4个角点，则可能是矩形
if len(approx) == 4:
        # 计算四边形的内部角度
        angles = []
        for i in range(4):
            pt1 = approx[i][0]
            pt2 = approx[(i + 1) % 4][0]
            pt3 = approx[(i + 2) % 4][0]
            angle = np.degrees(np.arctan2(pt3[1] - pt2[1], pt3[0] - pt2[0]) - np.arctan2(pt1[1] - pt2[1], pt1[0] - pt2[0]))
            angles.append(angle)

# 检查角度是否符合条件
            if all(angle >= 80 and angle <= 140 for angle in angles):
                for cnt in contours:
                # 计算每个轮廓的颜色均值
                    mask = np.zeros(gray.shape, np.uint8)
                    cv2.drawContours(mask, [cnt], -1, 255, -1)
                    mean_val = cv2.mean(image, mask=mask)
# 根据颜色均值判断色差大小
# 这里可以根据实际情况定义色差的阈值
                    if mean_val[0] < 100 and mean_val[1] < 100 and mean_val[2] < 100:
# 绘制或保留色差较小的轮廓
                        cv2.drawContours(image, [cnt], -1, (0, 255, 0), 2)
    # 获取矩形的坐标
                        x, y, w, h = cv2.boundingRect(approx)
                        x2, y2 = x + w, y + h

    # 计算矩形的中点坐标
                        center_x = (x + x2) // 2
                        center_y = (y + y2) // 2
    # 打印矩形的中点坐标
                        print("矩形中点坐标:", center_x, center_y)
    # 在图像上绘制中点
                        cv2.circle(image, (center_x, center_y), 5, (255, 0, 0), -1)
    # 打印矩形的坐标
                        print("矩形左上角坐标:", x, y)
                        print("矩形右下角坐标:", x2, y2)
